In [1]:
import numpy as np
import itertools
from tqdm import tqdm

# Quiver mutation and mutation sequences

Functions to do a single mutation and a sequence of mutations. We need the max_plus for this too

In [2]:
def max_plus(num):
    if num <= 0:
        return 0
    return num

def mutate(quiver, vertex):
    dim = quiver.shape[0]
    result = np.zeros(shape=(dim, dim))
    for i in range(dim):
        for j in range(dim):
            if (i == vertex or j == vertex):
                result[i,j] = -quiver[i,j]
            else:
                result[i,j] = quiver[i,j]+np.sign(quiver[i,vertex])*max_plus(quiver[i,vertex]*quiver[vertex,j])
    return np.array(result)

def mutation_sequence(quiver, sequence):
    current_quiver = quiver
    for vertex in sequence:
        current_quiver = mutate(current_quiver, vertex)
    return current_quiver

In [3]:
test1 = np.array([
    [ 0., -1.,  0.,  0.],
    [ 1.,  0., -1.,  0.],
    [ 0.,  1.,  0.,  1.],
    [ 0.,  0., -1.,  0.]
])

mutate(test1, 2)

array([[ 0., -1., -0.,  0.],
       [ 1.,  0.,  1.,  0.],
       [-0., -1., -0., -1.],
       [ 0.,  0.,  1.,  0.]])

In [4]:
mutation_sequence(test1, [1,2,1])

array([[ 0., -0.,  1.,  0.],
       [-0., -0.,  1., -1.],
       [-1., -1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.]])

# Permutations

A function to return all permutations which, applied to the first quiver, give the second. We can use the same quiver for both arguments to find permutation which fix it quiver

In [5]:
def which_permutation(quiver1, quiver2):
    dim = quiver1.shape[0]
    permutation_mats = list(itertools.permutations(np.eye(dim)))
    permutation_mats = [np.array(mat) for mat in permutation_mats]
    pairs = [[np.matmul(np.matmul(mat, quiver1), np.linalg.inv(mat)), mat] for mat in permutation_mats]
    return [pair[1] for pair in pairs if np.array_equal(pair[0], quiver2)]

In [6]:
test2 = np.array([
    [0, 1, -1, 0, -1, 1],
    [-1, 0, 2, -1, 1, -1],
    [1, -2, 0, 1, 1, -1],
    [0, 1, -1, 0, -1, 1],
    [1, -1, -1, 1, 0, 0],
    [-1, 1, 1, -1, 0, 0]
])

test3 = np.array([
    [ 0.,  1., -1., -1.,  1.,  0.],
    [-1.,  0.,  2.,  1., -1., -1.],
    [ 1., -2.,  0.,  1., -1.,  1.],
    [ 1., -1., -1.,  0.,  0.,  1.],
    [-1.,  1.,  1.,  0.,  0., -1.],
    [ 0.,  1., -1., -1.,  1.,  0.]
])

In [7]:
# Two permutations can be applied to test2 to give test3
which_permutation(test2, test3)

[array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.]]),
 array([[0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.]])]

In [8]:
# Two permutations can be applied to test2 to give itself (including the identity)
which_permutation(test2, test2)

[array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]])]

# Are quivers equivalent

In [9]:
def are_equivalent(quiver1, quiver2, depth, early_stop=True):
    '''
    We create a list of all possible permutations of quiver2. We then perform all possible sequences of mutations of quiver1
    with length <= the depth parameter, that give a permutation of quiver2, using our list. 
    early_stop=True will quit as soon as we find the first sequence that works. Otherwise we find all possibilities
    '''
    dim = quiver1.shape[0]
    permutation_mats = list(itertools.permutations(np.eye(dim)))
    permutation_mats = [np.array(mat) for mat in permutation_mats]
    quiver2_permutations = [{'permuted_quiver_2' : np.matmul(np.matmul(mat, quiver2), np.linalg.inv(mat)),'permutation_mat': mat} for mat in permutation_mats]
    
    total_check = (dim**depth-1) // (dim-1) # This is the number of seeds we actually check, as a function of the depth
    found_pairs = []
    pairs_to_check = [[quiver1, ""]]
    mutation_sequences = []
    for i in tqdm(range(1, total_check+1)):
        current_quiver = pairs_to_check[0][0] # Take the first quiver to check
        if any((current_quiver == quiver).all() for quiver in [quiver2_permutations[0]['permuted_quiver_2'] for quiver in quiver2_permutations]):
            if early_stop:
                return pairs_to_check[0][1]
            mutation_sequences.append(pairs_to_check[0][1])
        if not any((current_quiver == quiver).all() for quiver in [pair[0] for pair in found_pairs]):
            found_pairs.append(pairs_to_check[0])
        for j in range(0, dim): # Now we look at the neighbours of current_quiver
            next_quiver = mutate(current_quiver, j)
            if not any((next_quiver == quiver).all() for quiver in [pair[0] for pair in found_pairs] + [pair[0] for pair in pairs_to_check]):
                pairs_to_check.append([next_quiver, pairs_to_check[0][1]+str(j)+","])  
        pairs_to_check.pop(0) 
        if pairs_to_check == []:
            return "Ran out of quivers to check, so the pair of quivers are not mutation equivalent"
    if mutation_sequences:
        return mutation_sequences
    return "Quivers are not mutation equivalent at this depth, but possibly are at larger depths"

In [10]:
test4 = np.array([
    [0, 1, -1, 0, -1, 1],
    [-1, 0, 2, -1, 1, -1],
    [1, -2, 0, 1, 1, -1],
    [0, 1, -1, 0, -1, 1],
    [1, -1, -1, 1, 0, 0],
    [-1, 1, 1, -1, 0, 0]
])

test5 = np.array([
    [0, 0, -1, 1, 1, -1],
    [0, 0, 1, -1, -1, 1],
    [1, -1, 0, 0, -1, 1],
    [-1, 1, 0, 0, 1, -1],
    [-1, 1, 1, -1, 0, 0],
    [1, -1, -1, 1, 0, 0]
])

In [11]:
are_equivalent(test4, test5, 6)

 35%|███████████████████████████▋                                                   | 3277/9331 [02:52<05:17, 19.05it/s]


'0,2,1,5,3,5,'

In [12]:
# are_equivalent says that applying the following sequence to test4 should give test5 up to permutation.
mutation_sequence(test4, [0,2,1,5,3,5])

array([[ 0.,  0., -1.,  1.,  1., -1.],
       [ 0.,  0.,  1., -1., -1.,  1.],
       [ 1., -1.,  0.,  0., -1.,  1.],
       [-1.,  1.,  0.,  0.,  1., -1.],
       [-1.,  1.,  1., -1.,  0., -0.],
       [ 1., -1., -1.,  1., -0., -0.]])

In [13]:
# Actually in this case the permutation is the identity, which we can see by eye or by which_permutation
# this also gives the permuations that fix test5
which_permutation(mutation_sequence(test4, [0,2,1,5,3,5]), test5)

[array([[1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 array([[0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.]]),
 array([[0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
      